# Data Compressed - Powershell
An adversary may compress data (e.g., sensitive documents) that is collected prior to exfiltration in order to make it portable and minimize the amount of data sent over the network

## Rule Content
```
- title: Data Compressed - Powershell
  id: 6dc5d284-69ea-42cf-9311-fb1c3932a69a
  status: experimental
  description: An adversary may compress data (e.g., sensitive documents) that is
    collected prior to exfiltration in order to make it portable and minimize the
    amount of data sent over the network
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1002/T1002.yaml
  logsource:
    product: windows
    service: powershell
    description: Script block logging must be enabled
    category: null
  detection:
    selection:
      EventID: 4104
      keywords|contains|all:
      - -Recurse
      - '|'
      - Compress-Archive
    condition: selection
  falsepositives:
  - highly likely if archive ops are done via PS
  level: low
  tags:
  - attack.exfiltration
  - attack.t1002

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-powershell-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4104" AND keywords.keyword:*\-Recurse* AND keywords.keyword:*|* AND keywords.keyword:*Compress\-Archive*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()